In [54]:
%load_ext autoreload
%autoreload 2
from visualize import (
    shot_frames_navigator,
    animate_pitch,
    animate_chain,
    manual_labeling,
)
from data import load_and_process_soccer_events, load_and_process_metadata
from label import pos_labeling
from IPython.display import HTML
import polars as pl

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
tracking_file = f"/home/soccerdata/FIFA_WorldCup_2022/Tracking Data/{10516}.jsonl"
with open(tracking_file, "r") as tracking_data:
    print(type(tracking_data.tell()))
    print(tracking_data.readline())

<class 'int'>
{"version":"4.1.0","gameRefId":10516.0,"generatedTime":"2023-07-12T07:34:08.245914Z","smoothedTime":"2024-02-02T14:08:08.199111Z","videoTimeMs":179646.313,"frameNum":5384,"period":1,"periodElapsedTime":0.000313,"periodGameClockTime":0.000313,"homePlayers":[{"jerseyNum":"8","confidence":"HIGH","visibility":"VISIBLE","x":0.245,"y":-0.403},{"jerseyNum":"1","confidence":"LOW","visibility":"ESTIMATED","x":-36.836,"y":-0.63},{"jerseyNum":"24","confidence":"HIGH","visibility":"VISIBLE","x":1.243,"y":-24.731},{"jerseyNum":"18","confidence":"HIGH","visibility":"VISIBLE","x":0.142,"y":18.62},{"jerseyNum":"14","confidence":"HIGH","visibility":"VISIBLE","x":-15.28,"y":14.586},{"jerseyNum":"10","confidence":"HIGH","visibility":"VISIBLE","x":-8.305,"y":-5.152},{"jerseyNum":"2","confidence":"HIGH","visibility":"VISIBLE","x":-13.442,"y":-22.476},{"jerseyNum":"9","confidence":"HIGH","visibility":"VISIBLE","x":-17.737,"y":-7.91},{"jerseyNum":"20","confidence":"HIGH","visibility":"VISIBLE",

In [62]:
event_df, players_df = load_and_process_soccer_events("/home/soccerdata/FIFA_WorldCup_2022/Event Data")
metadata_df = load_and_process_metadata("/home/soccerdata/FIFA_WorldCup_2022/Metadata")

Game number 1 in progress...
Event number 6729792 in progress...
Event number 6729794 in progress...
Event number 6729823 in progress...
Event number 6729823 in progress...
Event number 6729797 in progress...
Event number 6729799 in progress...
Event number 6729801 in progress...
Event number 6729806 in progress...
Event number 6729808 in progress...
Event number 6729811 in progress...
Event number 6729815 in progress...
Event number 6729817 in progress...
Event number 6729886 in progress...
Event number 6729831 in progress...
Event number 6729837 in progress...
Event number 6729838 in progress...
Event number 6729842 in progress...
Event number 6729845 in progress...
Event number 6729845 in progress...
Event number 6729847 in progress...
Event number 6729848 in progress...
Event number 6729853 in progress...
Event number 6729855 in progress...
Event number 6729857 in progress...
Event number 6729860 in progress...
Event number 6729864 in progress...
Event number 6729877 in progress...

In [63]:
players_df

index,gameId,gameEventId,jerseyNum,x,y,z,velocity,direction,team,visibility
u32,i64,i64,str,f64,f64,f64,f64,f64,str,str
0,10516,6729792,"""8""",52.396,35.168,0.0,2.145447,-0.500441,"""home""",null
1,10516,6729792,"""1""",15.097,33.486,0.0,0.884801,-0.374334,"""home""",null
2,10516,6729792,"""24""",46.035,11.092,0.0,0.723434,1.107149,"""home""",null
3,10516,6729792,"""18""",52.836,54.51,0.0,3.751009,-0.498721,"""home""",null
4,10516,6729792,"""14""",38.326,48.43,0.0,2.225404,-0.51095,"""home""",null
…,…,…,…,…,…,…,…,…,…,…
31298,10516,6731536,"""8""",50.491,27.187,0.0,1.08627,-2.615526,"""away""",null
31299,10516,6731536,"""10""",54.396,41.534,0.0,0.644251,2.422763,"""away""",null
31300,10516,6731536,"""24""",42.884,25.814,0.0,1.092591,-2.8023,"""away""",null


In [64]:
players_df.select(pl.max('velocity'))

velocity
f64
210.507091


In [65]:
players_df.filter(pl.col('velocity') > 50)

index,gameId,gameEventId,jerseyNum,x,y,z,velocity,direction,team,visibility
u32,i64,i64,str,f64,f64,f64,f64,f64,str,str
7106,10516,6730332,null,49.58,31.88,0.52,50.413351,1.109837,null,"""VISIBLE"""
21389,10516,6730129,null,72.46,57.6,1.43,56.517386,-2.947352,null,"""ESTIMATED"""
22930,10516,6730510,null,83.6,59.65,0.41,82.469907,-1.478806,null,"""ESTIMATED"""
27668,10516,6730708,null,14.76,23.23,3.61,210.507091,1.323784,null,"""ESTIMATED"""
30980,10516,6731493,null,39.8,54.05,0.04,54.91894,-0.762675,null,"""ESTIMATED"""


In [27]:
players_df.filter(pl.col('team').is_null())

index,gameId,gameEventId,jerseyNum,x,y,z,velocity,direction,team
u32,i64,i64,str,f64,f64,f64,f64,f64,str
22,10516,6729792,null,52.4,35.17,0.27,11.709429,2.939271,null
45,10516,6729794,null,40.61,35.18,0.72,5.462956,-0.982794,null
68,10516,6729823,null,47.54,24.83,2.73,26.197666,-1.050215,null
91,10516,6729823,null,47.64,24.68,2.73,20.750386,-1.067953,null
114,10516,6729797,null,60.59,0.77,2.12,null,null,null
…,…,…,…,…,…,…,…,…,…
31210,10516,6731527,null,92.78,58.22,4.7,9.69697,-1.570796,null
31233,10516,6731528,null,92.33,40.64,3.44,19.928707,-0.490852,null
31256,10516,6731528,null,92.32,40.32,3.44,17.881356,0.016948,null


In [ ]:
pos_chains = pos_labeling(event_df, chain_len=0)

In [ ]:
shot_frames_navigator(pos_chains[31], event_df, players_df, "/home/soccerdata/frames",show_video=True)

In [ ]:
anim = animate_pitch(pos_chains[1400], event_df, players_df, metadata_df, interval=500)
HTML(anim.to_jshtml())

In [ ]:
manual_labels = manual_labeling(
    (0, 2), animate_chain, pos_chains, event_df, players_df, metadata_df
)